<a href="https://colab.research.google.com/github/ecazar/ProyectoIIB/blob/main/ProjectIIB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Codificación e indexación multimodal del corpus

## Utilizar el corpus Consumer Reviews of Amazon Products (disponible en Kaggle) que contiene:
* imágenes de productos,
* escripciones textuales (título y/o descripción),
* información adicional como categorías, atributos o reseñas (si están disponibles).


nos conectamos al drive para guardar archivos importantes y no tener q volver a entrenarlos o limpiarlos

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


cargamos el dataset de **Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products** con las columnas necesarias para el proyecto






In [2]:
import kagglehub
import pandas as pd
path = kagglehub.dataset_download("datafiniti/consumer-reviews-of-amazon-products")
df = pd.read_csv(path + "/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv",
                usecols=['name','categories','imageURLs','reviews.doRecommend','reviews.text','reviews.title'])

Using Colab cache for faster access to the 'consumer-reviews-of-amazon-products' dataset.


separamos las columnas para diferenciar entre la descripcion del producto, la imagen y las reviews

In [3]:
df_text = df['name'] + " " + df['categories']
df_image = df['imageURLs']
df_review = df['reviews.title'] + " " + df['reviews.text']
df_recommend = df['reviews.doRecommend']

preprocesamos el texto con ayuda de la gpu con la libreria spacy

In [4]:
df_text = df['name'] + " " + df['categories']
import spacy
activated = spacy.prefer_gpu() # uso de gpu para el preprocesamiento
nlp = spacy.load("en_core_web_sm")

docs = nlp.pipe(df_text, batch_size=5000)
df_text = [" ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct]) for doc in docs]
df_text = pd.Series(df_text)

docs = nlp.pipe(df_text, batch_size=5000)
df_review = [" ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct]) for doc in docs]
df_review = pd.Series(df_review)

In [5]:
def check_first_valid_image(urls, timeout=5):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    for url in map(str.strip, str(urls).split(",")):
        if not url.startswith("http"):
            continue
        try:
            response = requests.head(url, timeout=timeout, headers=headers, allow_redirects=True)
            if response.status_code == 200:
                content_type = response.headers.get('Content-Type', '')
                if 'image' in content_type:
                    return url
        except Exception:
            continue
    return None
# df['imageURLs'] = df_image.apply(check_first_valid_image)

In [6]:
# df.to_csv('/content/drive/MyDrive/proyectoRI/df_limpio.csv', index=False)

In [7]:
df = pd.read_csv('/content/drive/MyDrive/proyectoRI/df_limpio.csv')

###  Codificar textos e imágenes en espacios vectoriales utilizando modelos de embeddings multimodales (por ejemplo, CLIP u otros encoders equivalentes).

importamos las librerias necesaria para crear los embeddings

In [8]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
from PIL import Image
import requests
import numpy as np
from io import BytesIO
import gc
from tqdm.notebook import tqdm # progress bar

model = SentenceTransformer('clip-ViT-B-32')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


1. Creacion de embeddings para la parte de la descripcion del producto

In [9]:
# ls_text = df_text.tolist()
# embeddings_text = model.encode(ls_text, batch_size=128, show_progress_bar=True) # uso de gpu
# np.save('/content/drive/MyDrive/proyectoRI/emb_txt.npy', embeddings_text)

2. Creacion de embeddings para la parte de las imagenes, donde descarga las imagenes y las entrena

In [10]:
# def download_first_valid_image(urls, timeout=5):
#     headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
#     for url in map(str.strip, str(urls).split(",")):
#         if not url.startswith("http"):
#             continue
#         try:
#             response = requests.get(url, timeout=timeout, headers=headers)
#             response.raise_for_status()
#             img = Image.open(BytesIO(response.content)).convert("RGB")
#             img = img.resize((224, 224), Image.Resampling.LANCZOS)
#             return img
#         except Exception:
#             continue
#     return None

# chunk_size = 250
# total_embeddings = []
# indices_validos = []
# reporte_fallos = []

# pbar = tqdm(total=len(df_image), desc="Buscando imagen válida por producto")

# for start in range(0, len(df_image), chunk_size):
#     batch_urls = df_image[start : start + chunk_size]
#     batch_images = []
#     exitos_en_este_batch = 0

#     for offset, url_block in enumerate(batch_urls):
#         idx = start + offset
#         img = download_first_valid_image(url_block)

#         if img is not None:
#             batch_images.append(img)
#             indices_validos.append(idx)
#             exitos_en_este_batch += 1
#         else:
#             reporte_fallos.append({
#                 "index": idx,
#                 "error": "Ninguna URL funcionó"
#             })
#         pbar.update(1)

#     if batch_images:
#         embeddings = model.encode(
#             batch_images,
#             batch_size=64,
#             show_progress_bar=False
#         )
#         total_embeddings.append(embeddings)

#     # LIMPIEZA RAM
#     del batch_images
#     gc.collect()

# pbar.close()

# # Unimos todo al final con tus nombres
# embeddings_images = np.vstack(total_embeddings)
# np.save('/content/drive/MyDrive/proyectoRI/emb_img.npy', embeddings_images)

cargamos los modelos guardados en el drive, si es que tenemos

In [11]:
embeddings_images = np.load('/content/drive/MyDrive/proyectoRI/emb_img.npy')
embeddings_text = np.load('/content/drive/MyDrive/proyectoRI/emb_txt.npy')

## Construir uno o más índices vectoriales (por ejemplo, FAISS o ChromaDB) para soportar búsquedas eficientes.

importamos las librerias necesarios y creamos un cliente que guarda los datos en el drive


In [12]:
!pip install chromadb
import chromadb
from chromadb.config import Settings
chroma_client = chromadb.PersistentClient(path="/content/drive/MyDrive/proyectoRI/chroma_db")

creamos dos colecciones una para la descripcion de los productos y otra para las imagenes de los productos

In [13]:
collection_text = chroma_client.create_collection(
    name="products_text"
)
collection_image = chroma_client.create_collection(
    name="products_image"
)

creamos los ids del dataset

In [14]:
ids = df.index.astype(str).tolist()

creamos metadatos por motivo de que se necesite el uso de las reviews para mejorar la recuperacion de la busqueda

In [15]:
metadatas = [
    {
        "id": str(i),
        "name": df.loc[i, "name"],
        "categories": df.loc[i, "categories"],
        "imageURL": df.loc[i, "imageURLs"],
        "reviews": str(df.loc[i, "reviews.title"] or "") + " " + str(df.loc[i, "reviews.text"] or ""),
        "doRecommend": str(df.loc[i, "reviews.doRecommend"])
    }
    for i in df.index
]


Guarda estos productos, con estos vectores y estos datos

In [16]:
collection_text.add(
    ids=ids,
    embeddings=embeddings_text,
    documents=df_text.tolist(),
    metadatas=metadatas
)
collection_image.add(
    ids=ids,
    embeddings=embeddings_images,
    documents=df_image.tolist(),
    metadatas=metadatas
)


carga las colecciones guardadas en el drive

In [17]:
collection_text = chroma_client.get_collection(name="products_text")
collection_image = chroma_client.get_collection(name="products_image")

# Búsqueda multimodal (retrieval inicial)

In [18]:
import matplotlib.pyplot as plt
import requests
from PIL import Image
from io import BytesIO

Permitir al usuario realizar consultas mediante
* texto (text-to-product),
* imagen (image-to-product).

o ambos, para que busque por imagen y texto a la vez, por medio de $$\vec{v}_{query} = \frac{0.7 \cdot \vec{v}_{text} + 0.3 \cdot \vec{v}_{image}}{2}$$

dandole mas importancia al texto que a la imagen al momento de buscar

In [19]:
import torch
from PIL import Image

def search_products(texto_query=None, imagen_query=None, top_k=5):
    queries_combinated = []
    if texto_query:
        query_vector = model.encode(texto_query, convert_to_numpy=True)
        queries_combinated.append(query_vector)
    if imagen_query:
        img = Image.open(imagen_query) if isinstance(imagen_query, str) else imagen_query
        query_vector = model.encode(img, convert_to_numpy=True)
        queries_combinated.append(query_vector)
    if len(queries_combinated) == 2:
        query_vector = (0.7*queries_combinated[0] + 0.3*queries_combinated[1]) / 2
    else:
        query_vector = queries_combinated[0]
    results = collection_image.query(
        query_embeddings=query_vector.tolist(),
        n_results=top_k
    )
    return format_results(results)

def format_results(results):
    candidates = []
    for i in range(len(results['ids'][0])):
        candidates.append({
            "id": results['ids'][0][i],
            "score": results['distances'][0][i],
            "metadata": results['metadatas'][0][i]
        })
    return candidates

def mostrar_resultados(candidates):
    print("\n" + "="*50)
    print(f"🔍 RESULTADOS DEL RETRIEVAL INICIAL (Top {len(candidates)})")
    print("="*50 + "\n")
    if not candidates:
        print("❌ No se encontraron productos.")
        return
    for i, c in enumerate(candidates, 1):
        meta = c['metadata']
        id_prod = c.get('id', 'N/A')
        titulo = meta.get('name', 'Sin título')
        categoria = meta.get('categories', 'N/A')
        score = c.get('score', 0.0)
        fuente_img = meta.get('imageURL') or "No disponible"
        print(f"{i}. [ID: {id_prod}] - DISTANCIA: {score:.4f}")
        print(f"   📦 TÍTULO:    {titulo}")
        print(f"   🏷️  CATEGORÍA: {categoria}")
        print(f"   🖼️  IMAGEN:    {fuente_img}")
        print("-" * 50)

### Mostrar los resultados recuperados junto con su información básica (imagen, título, categoría,etc.).
1. text-to-product

In [31]:
print("--- Buscando solo por texto ---")
res_texto = search_products(texto_query="pink table")
mostrar_resultados(res_texto)

--- Buscando solo por texto ---

🔍 RESULTADOS DEL RETRIEVAL INICIAL (Top 5)

1. [ID: 4903] - DISTANCIA: 135.3627
   📦 TÍTULO:    Fire Tablet with Alexa, 7" Display, 16 GB, Magenta - with Special Offers
   🏷️  CATEGORÍA: Tablets,Fire Tablets,Electronics,iPad & Tablets,Android Tablets,Computers & Tablets,All Tablets
   🖼️  IMAGEN:    https://images-na.ssl-images-amazon.com/images/I/510i3yBUrsL.jpg
--------------------------------------------------
2. [ID: 4904] - DISTANCIA: 135.3627
   📦 TÍTULO:    Fire Tablet with Alexa, 7" Display, 16 GB, Magenta - with Special Offers
   🏷️  CATEGORÍA: Tablets,Fire Tablets,Electronics,iPad & Tablets,Android Tablets,Computers & Tablets,All Tablets
   🖼️  IMAGEN:    https://images-na.ssl-images-amazon.com/images/I/510i3yBUrsL.jpg
--------------------------------------------------
3. [ID: 4906] - DISTANCIA: 135.3627
   📦 TÍTULO:    Fire Tablet with Alexa, 7" Display, 16 GB, Magenta - with Special Offers
   🏷️  CATEGORÍA: Tablets,Fire Tablets,Electronics,i

2. image-to-product

In [34]:
print("\n--- Buscando solo por imagen ---")
res_imagen = search_products(imagen_query="/content/image.jpeg")
mostrar_resultados(res_imagen)


--- Buscando solo por imagen ---

🔍 RESULTADOS DEL RETRIEVAL INICIAL (Top 5)

1. [ID: 1978] - DISTANCIA: 99.5242
   📦 TÍTULO:    Fire Tablet, 7 Display, Wi-Fi, 16 GB - Includes Special Offers, Black
   🏷️  CATEGORÍA: Fire Tablets,Computers/Tablets & Networking,Tablets,All Tablets,Amazon Tablets,Frys,Computers & Tablets,Tablets & eBook Readers
   🖼️  IMAGEN:    http://i.ebayimg.com/thumbs/images/g/3MIAAOSwol5Y4W6z/s-l96.jpg
--------------------------------------------------
2. [ID: 2710] - DISTANCIA: 99.5242
   📦 TÍTULO:    Fire Tablet, 7 Display, Wi-Fi, 16 GB - Includes Special Offers, Black
   🏷️  CATEGORÍA: Fire Tablets,Computers/Tablets & Networking,Tablets,All Tablets,Amazon Tablets,Frys,Computers & Tablets,Tablets & eBook Readers
   🖼️  IMAGEN:    http://i.ebayimg.com/thumbs/images/g/3MIAAOSwol5Y4W6z/s-l96.jpg
--------------------------------------------------
3. [ID: 2732] - DISTANCIA: 99.5242
   📦 TÍTULO:    Fire Tablet, 7 Display, Wi-Fi, 16 GB - Includes Special Offers, Black


3. (text+image)-to-product

In [33]:
print("\n--- Búsqueda (Texto + Imagen) ---")
res_mix = search_products(
    texto_query="pink color",
    imagen_query="/content/image.jpeg"
)
mostrar_resultados(res_mix)



--- Búsqueda (Texto + Imagen) ---

🔍 RESULTADOS DEL RETRIEVAL INICIAL (Top 5)

1. [ID: 4903] - DISTANCIA: 82.6969
   📦 TÍTULO:    Fire Tablet with Alexa, 7" Display, 16 GB, Magenta - with Special Offers
   🏷️  CATEGORÍA: Tablets,Fire Tablets,Electronics,iPad & Tablets,Android Tablets,Computers & Tablets,All Tablets
   🖼️  IMAGEN:    https://images-na.ssl-images-amazon.com/images/I/510i3yBUrsL.jpg
--------------------------------------------------
2. [ID: 4906] - DISTANCIA: 82.6969
   📦 TÍTULO:    Fire Tablet with Alexa, 7" Display, 16 GB, Magenta - with Special Offers
   🏷️  CATEGORÍA: Tablets,Fire Tablets,Electronics,iPad & Tablets,Android Tablets,Computers & Tablets,All Tablets
   🖼️  IMAGEN:    https://images-na.ssl-images-amazon.com/images/I/510i3yBUrsL.jpg
--------------------------------------------------
3. [ID: 4909] - DISTANCIA: 82.6969
   📦 TÍTULO:    Fire Tablet with Alexa, 7" Display, 16 GB, Magenta - with Special Offers
   🏷️  CATEGORÍA: Tablets,Fire Tablets,Electronics,i

# Re-ranking de resultados